---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    #university_towns_df = pd.read_table('university_towns.txt',delimiter='(',header=None,skip_blank_lines=True,
                                        #names=['Town','Univeristy','State'])
    #del university_towns_df['Univeristy']
    #university_towns_df['RegionName']=university_towns_df['Town']
    #university_towns_df.set_index('Town')
    
    #t_states = university_towns_df.where(university_towns_df['Town'].str.contains("\[")).dropna(how='all')
    #t_states = t_states['Town'].str.split('\[')
    #t_states = pd.DataFrame(t_states.str[0])
    #t_states.rename(columns={'Town':'State'},inplace=True)
    #t_states.set_index('State',inplace=True)
    
    #states_df = pd.DataFrame(columns=['Code','State'])
    #states_series = pd.Series(states)
    #states_series = states_series.reset_index()
    #states_df['Code'] = states_series['index']
    #states_df['State'] = states_series[0]
        
    #states_df = states_df.set_index('State')
    
    #merge_df = pd.merge(t_states,states_df,left_index=True,right_index=True, how='inner')
    #for itr in university_towns_df.itertuples():        
        #print(itr[0])
        #university = np.str(itr[2]).split(')')[0]
        #if(itr[1].find("edit")>=0):
            #temp_state = itr[1].replace('[edit]','')
        #else:
            #university_towns_df['State'][itr[0]]=temp_state
            #university_towns_df['Univeristy'][itr[0]]=university
                   
    #3university_towns_df.dropna(inplace=True)      
    #university_towns_df.set_index(['State'],inplace=True)
    #del university_towns_df['Town']
    #return  university_towns_df
    with open('university_towns.txt') as file:
        data = []
        for line in file:
            data.append(line[:-1])
    state_town = []
    for line in data:
        if line[-6:] == '[edit]':
            state = line[:-6]
        elif '(' in line:
            town = line[:line.index('(')-1]
            state_town.append([state,town])
        else:
            town = line
            state_town.append([state,town])
    state_college_df = pd.DataFrame(state_town,columns = ['State','RegionName'])
    return state_college_df
        

    
get_list_of_university_towns()

State                    RegionName
0          Alabama                        Auburn
1          Alabama                      Florence
2          Alabama                  Jacksonville
3          Alabama                    Livingston
4          Alabama                    Montevallo
5          Alabama                          Troy
6          Alabama                    Tuscaloosa
7          Alabama                      Tuskegee
8           Alaska                     Fairbanks
9          Arizona                     Flagstaff
10         Arizona                         Tempe
11         Arizona                        Tucson
12        Arkansas                   Arkadelphia
13        Arkansas                        Conway
14        Arkansas                  Fayetteville
15        Arkansas                     Jonesboro
16        Arkansas                      Magnolia
17        Arkansas                    Monticello
18        Arkansas                  Russellville
19        Arkansas                        Searcy
20      California                        Angwin
21      California                        Arcata
22      California                      Berkeley
23      California                         Chico
24      California                     Claremont
25      California                        Cotati
26      California                         Davis
27      California                        Irvine
28      California                    Isla Vista
29      California  University Park, Los Angeles
..             ...                           ...
487       Virginia                          Wise
488       Virginia                    Chesapeake
489     Washington                    Bellingham
490     Washington                        Cheney
491     Washington                    Ellensburg
492     Washington                       Pullman
493     Washington  University District, Seattle
494  West Virginia                        Athens
495  West Virginia                    Buckhannon
496  West Virginia                      Fairmont
497  West Virginia                     Glenville
498  West Virginia                    Huntington
499  West Virginia                    Montgomery
500  West Virginia                    Morgantown
501  West Virginia                 Shepherdstown
502  West Virginia                  West Liberty
503      Wisconsin                      Appleton
504      Wisconsin                    Eau Claire
505      Wisconsin                     Green Bay
506      Wisconsin                     La Crosse
507      Wisconsin                       Madison
508      Wisconsin                     Menomonie
509      Wisconsin                     Milwaukee
510      Wisconsin                       Oshkosh
511      Wisconsin                   Platteville
512      Wisconsin                   River Falls
513      Wisconsin                 Stevens Point
514      Wisconsin                      Waukesha
515      Wisconsin                    Whitewater
516        Wyoming                       Laramie

[517 rows x 2 columns]

In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp_df = pd.read_excel('gdplev.xls',skiprows=5)
    del gdp_df['Unnamed: 3']
    del gdp_df['Unnamed: 7']
    gdp_df.rename(columns={'Unnamed: 0':'Year','Unnamed: 4':'Quarter'},inplace=True)
    gdp_quarterly_df = gdp_df[['Quarter','GDP in billions of current dollars.1','GDP in billions of chained 2009 dollars.1']]
    #gdp_quarterly_df.set_index(['Quarter'],inplace=True)
    gdp_quarterly_df = gdp_quarterly_df.dropna()
    #year = (gdp_quarterly_df.index.str.split('q'))
    #gdp_quarterly_df['Year'] = year.str[0]
    gdp_quarterly_df['Diff'] = gdp_quarterly_df['GDP in billions of chained 2009 dollars.1'].diff()
    #gdp_quarterly_df =  gdp_quarterly_df.where(gdp_quarterly_df['Diff']<0).dropna()           
    #gdp_quarterly_df['Rank'] = gdp_quarterly_df.groupby('Year')['Diff'].rank()
    
    #last_to_last_diff = last_diff = 0
    #recession_quarter = ''
    #count = 0
    #recession_start = pd.DataFrame()
    #print(gdp_quarterly_df['Diff'].iloc[0])
    quarters = []
    for i in range(len(gdp_quarterly_df) - 3):
        #if (gdp_quarterly_df.iloc[i][1] > gdp_quarterly_df.iloc[i+1][1]) & (gdp_quarterly_df.iloc[i+1][1] > gdp_quarterly_df.iloc[i+2][1]):
        if ((gdp_quarterly_df['Diff'].iloc[i]<0)& (gdp_quarterly_df['Diff'].iloc[i+1]<0)): 
            if((gdp_quarterly_df['Diff'].iloc[i+3]<0)):
                quarters.append((gdp_quarterly_df['Quarter'].iloc[i]))
            else:
                quarters.append((gdp_quarterly_df['Quarter'].iloc[i]))
                            
    return quarters

   
(get_recession_start())

['1947q2',
 '1949q1',
 '1953q3',
 '1953q4',
 '1957q4',
 '1969q4',
 '1974q3',
 '1974q4',
 '1980q2',
 '1981q4',
 '1990q4',
 '2008q3',
 '2008q4',
 '2009q1']

In [28]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp_df = pd.read_excel('gdplev.xls',skiprows=5)
    del gdp_df['Unnamed: 3']
    del gdp_df['Unnamed: 7']
    gdp_df.rename(columns={'Unnamed: 0':'Year','Unnamed: 4':'Quarter'},inplace=True)
    gdp_quarterly_df = gdp_df[['Quarter','GDP in billions of current dollars.1','GDP in billions of chained 2009 dollars.1']]
    gdp_quarterly_df.set_index(['Quarter'],inplace=True)
    gdp_quarterly_df = gdp_quarterly_df.dropna()
    year = (gdp_quarterly_df.index.str.split('q'))
    gdp_quarterly_df['Year'] = year.str[0]
    gdp_quarterly_df['Diff'] = gdp_quarterly_df['GDP in billions of current dollars.1'].diff()
    recession_start = pd.Series(get_recession_start()) 
    recession_df = pd.DataFrame(columns=('Quarter','R_flag'))
    
    index = 0
    for item in recession_start:
        #print(type((item)))
        recession_df.set_value(index,'Quarter',item)
        recession_df.set_value(index,'R_flag',1)
        index+=1
    
    recession_df.set_index('Quarter',inplace=True)    
    final_df = pd.merge(gdp_quarterly_df,recession_df,how='left',left_index=True,right_index=True)    
    
    recession_end = pd.Series()
    next_to_next_diff = next_diff = 0
    end_quarter = ''
    recession_quarter = 0
    count = 0
    for itr in final_df.itertuples():
        #print(itr)
        if((itr[5]==1)):
            recession_quarter = itr[5]
            #print(recession_quarter)
       
        if((itr[4]>0) & (next_diff > 0) & (count<=2) ):
                next_to_next_diff = next_diff
                next_diff = itr[4]
                end_quarter = itr[0]
                count+=1
                print(next_diff)
        elif((itr[4]>0) & (count<=2) &(recession_quarter==1)):
                next_diff = itr[4]
                count+=1
                print(end_quarter)
        elif((count>2) & (next_to_next_diff>0) & (next_diff>0)): 
                print(end_quarter)
                count=0
                recession_quarter = 0   
                end_quarter = ''
              
    return recession_end
get_recession_end()


10.2
9.5
1950q2
11.8
16.1
8.1
1951q2
4.8
3.6
1.2
1952q2
13.1
7.3
3.8
1953q2
0.8
4.9
8.7
1954q4
8.4
8.7
6.9
1955q4
6.3
5.2
9.3
1956q4
2.2
7.5
4.4
1958q2
13.7
10.7
13.1
1959q2
4.1
14.0
3.3
1960q3
4.8
11.5
10.8
1961q3
13.6
7.4
7.0
1962q3
9.6
9.1
13.2
1963q3
16.3
9.7
12.0
1964q3
20.8
13.2
17.8
1965q3
24.2
9.9
13.6
1966q3
11.1
5.1
15.5
1967q3
27.9
25.2
16.0
1968q3
25.3
16.0
20.6
1969q3
12.8
16.6
18.4
1970q3
46.3
21.6
20.9
1971q3
40.2
36.3
23.7
1972q3
48.7
36.9
19.2
1973q3
15.6
39.5
29.2
1974q3
16.6
36.8
57.4
1975q3
58.6
32.4
33.6
1976q3
54.1
67.7
62.2
1977q3
40.0
127.9
62.3
1978q3
49.4
64.3
74.5
1979q3
65.8
3.4
60.1
1980q3
138.3
35.5
93.9
1981q3
57.5
35.8
40.7
1982q4
103.5
108.5
103.8
1983q4
102.2
72.4
60.2
1984q4
65.3
92.3
58.5
1985q4
38.9
64.4
49.8
1986q4
85.3
79.0
122.2
1987q4
117.1
91.8
113.2
1988q4
101.0
83.2
51.8
1989q4
83.9
54.8
31.6
1991q1
74.8
60.9
101.5
1992q1
94.2
111.1
50.6
1993q1
74.6
128.6
103.5
1994q1
82.5
124.4
68.6
1995q1
101.6
93.0
93.6
1996q1
97.5
128.1
115.0
1997q1
139.

Series([], dtype: float64)

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return "ANSWER"

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"